# Fast Infrastructure Assessment
- Author: Aidan Marlin
- Date: 20220606
- Version: 1.0
- Description: Jupyter Notebook to automate the execution of key OSINT processes

In [ ]:
target = "www.bing.com"
port = 443
ffuf_subdomain_target = "FUZZ.bing.com"
subfinder_target = "bing.com"

### No need to modify from here
ffuf_subdomain_target_https = "https://" + ffuf_subdomain_target
ffuf_subdomain_target_http = "http://" + ffuf_subdomain_target
ffuf_directory_brute = target + "/FUZZ"
http_target = "http://" + target
https_target = "https://" + target

!mkdir -p tools/ffuf_subdomain
!mkdir -p tools/ffuf_directory
!mkdir -p tools/nmap
!mkdir -p tools/puppeteer
!mkdir -p tools/subfinder
!mkdir -p tools/testssl


# Target

In [ ]:
print(target)


# Pentester Details

## IP Address

In [ ]:
import os
os.environ["PATH"] = os.environ["PATH"] + ":/run/current-system/sw/bin"

!curl https://ifconfig.me/


## Start Time

In [ ]:
!date


## Interface Info

In [ ]:
!ip a


# Web

## Screenshot

In [ ]:
os.system("docker run -u root --shm-size 1G --rm -v " + os.getcwd() + "/tools/puppeteer:/screenshots alekzonder/puppeteer:latest screenshot " + https_target + ":" + str(port) + " 1024x768")


![Target](tools/puppeteer/screenshot_1024_768.png)


## WhatWeb

In [ ]:
!docker run --rm guidelacour/whatweb ./whatweb {https_target}:{port}


# Fuzzing

## Subfinder

In [ ]:
!subfinder -d {subfinder_target} -o tools/subfinder/results.txt


## ffuf (subdomains)

In [ ]:
!echo ffuf -s -of all -w /home/user/git/wordlists/SecLists/Discovery/DNS/subdomains-top1million-5000.txt -u https://{ffuf_subdomain_target} -od tools/ffuf_subdomain
!echo "`find tools/ffuf_subdomain -type f | wc -l` responses worth investigating.."
!grep "Host: " tools/ffuf_subdomain -R


## ffuf (directory brute)


In [ ]:
!ffuf -ac -sf -of all -w /home/user/git/wordlists/SecLists/Discovery/Web-Content/raft-large-directories.txt -u https://{ffuf_directory_brute}:{port} -od tools/ffuf_directory
!echo "`find tools/ffuf_directory -type f | wc -l` responses worth investigating.."
!find tools/ffuf_directory -type f -exec grep -hE 'GET |^HTTP/' {} \; -exec echo \;


# Scanning

## TLS Scan

In [ ]:
!testssl.sh {target}:{port}


## Port Scan (TCP)

In [ ]:
!nmap -A -T4 {target} -oA tools/nmap/output


## Port Scan (UDP)

In [ ]:
!sudo nmap -sU -A -T4 {target} -oA tools/nmap/output

# End Time

In [ ]:
!date